### Parameter Estimation using ASSISTments Skill Builder Dataset

In [2]:
import pandas as pd
import json

import autograd.numpy as np
import autograd.scipy as ascipy
from autograd import value_and_grad
from autograd import grad
from scipy.optimize import minimize

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [4]:
# ASSISTment Data
df = pd.DataFrame()
data = pd.read_csv('../skill_builder_data.csv')

/Users/rishabh/.virtualenvs/ml/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# prerequistes
f = open('prereq.json')
prereqs = json.load(f)


In [6]:
# getting student 78178
per_student = data.loc[data['user_id'] == 78178]
per_student = per_student[pd.notnull(per_student['skill_name'])]
per_student_frame = per_student.sort_values(by=['order_id'])
# per_student_frame = per_student_frame[(per_student_frame['skill_name'] == 'Addition and Subtraction Fractions') 
#                                       | (per_student_frame['skill_name'] == 'Order of Operations All')]
per_student_frame = per_student_frame.sort_values(by=['order_id'])

n_skills = len(per_student_frame['skill_name'].unique())

In [7]:
def initialize(n):
    # initialize weight vector
    weight_vector = np.random.rand(n+1)

    # alpha (probability) vector
    alpha_vector = np.zeros(n+1)
    
    # initialize takeaway dictionary
    f_s = {}
    counter_f_s = {}
    counter = 0
    for each in per_student_frame['skill_name'].unique():
        f_s[each] = 0.0
        counter_f_s[each] = counter
        counter += 1
    return alpha_vector, f_s, counter_f_s

In [8]:
'''
Sigmoid function
1/(1+e^{-w'x})
'''
def sigmoid_function(weight, x):
    return 1/(1 + np.exp(-np.dot(weight, x)))

'''
Update Rule for Takeaway Fraction
F_s(T_i, n) = F_s(T_i, n-1) + (1-F_s[T_i, n-1])*alpha(T_i, n)
'''
def update_f_s(f_s, skill_name, alpha):
    f_s[skill_name] = f_s[skill_name] + (1-f_s[skill_name])*alpha
    return f_s

In [9]:
def construct_feature_vector(n, f_s, counter_f_s, skill_name, prereqs):
    # initilalize feature vector
    feature_vector = np.zeros(n+1)
    feature_vector[n] = 1
    
    # add F_s(skill_name)
    feature_vector[counter_f_s[skill_name]] = f_s[skill_name]
    
    # add F_s for all prereqs
    for each_skill in prereqs[skill_name]:
        if(each_skill in f_s):
            feature_vector[counter_f_s[each_skill]] = f_s[each_skill]
    return feature_vector

In [20]:
'''
Student Updates.
'''
def update_student(weight_vector, alpha_vector, f_s, counter_f_s):
    step = 1
    for index, row in per_student_frame.iterrows():
        skill_name = row['skill_name']
        if(skill_name not in f_s): continue
        skill_index = counter_f_s[skill_name]

        feature_vector = construct_feature_vector(n_skills, f_s, counter_f_s, skill_name, prereqs)
        alpha_vector[skill_index] = sigmoid_function(weight_vector, feature_vector)
        
        f_s = update_f_s(f_s, skill_name, alpha_vector[skill_index])

#         feature_vector[skill_index] = f_s[skill_name]
        step += 1
    return f_s

'''
Log Likelihood.
'''
def log_likelihood(weight_vector, sign=1.0):
    alpha_vector, f_s, counter_f_s = initialize(n_skills)
    all_f_s = update_student(weight_vector, alpha_vector, f_s, counter_f_s)
    s = 1
    for each in all_f_s.values():
        s = s + each
    return np.log(s)

def all_students_likelihood(weight_vector, sign=1.0):
    ids = data['user_id'].unique()
    

In [21]:
w0 = np.zeros(n_skills+1)

# alphas = log_likelihood(w0, alpha_vector, f_s, counter_f_s, alphas)
opt = minimize(log_likelihood, w0, method='CG', options={'disp':True})

Optimization terminated successfully.
         Current function value: 0.000004
         Iterations: 7
         Function evaluations: 1496
         Gradient evaluations: 44


In [22]:
opt.x

array([ -2.42418115e-01,  -2.12256314e-02,  -1.41444867e+00,
        -2.42418115e-01,  -1.08793902e-04,  -1.93933994e-01,
        -7.75735974e-01,  -1.01327896e-05,  -1.74908370e-01,
         0.00000000e+00,  -2.12256314e-02,  -1.91142608e-04,
        -1.93933994e-01,  -8.24592338e-01,  -3.13780038e-05,
        -1.93933994e-01,  -2.44610876e-03,  -3.13780038e-05,
        -2.06055343e-01,  -1.66501309e-03,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -3.76021490e-06,
        -1.14494091e-03,  -1.21263863e-02,  -3.14819044e-05,
        -1.14618771e-03,  -2.08729529e-03,  -1.26768660e-04,
        -1.26768660e-04,  -1.85617777e+01])

In [15]:
feature_vector = np.zeros(n_skills+1)
feature_vector[n_skills] = 1.
w = opt.x

In [16]:
alpha_vector, f_s, counter_f_s = initialize(n_skills)

In [19]:
a = opt.x
a
for k,v in counter_f_s.items():
    print(k, a[v])


('Write Linear Equation from Graph', -0.012126386282034218)
('Percent Of', 0.0)
('Proportion', 0.0)
('Finding Percents', -3.1378003768622875e-05)
('Multiplication and Division Integers', 0.0)
('Algebraic Solving', -0.021225631353445351)
('Solving Inequalities', -0.00012676866026595235)
('Percents', -0.82459233759436756)
('Ordering Fractions', -3.1378003768622875e-05)
('Exponents', -0.001146187714766711)
('Addition and Subtraction Fractions', -0.19393399357795715)
('Absolute Value', -0.021225631353445351)
('Number Line', -0.00012676866026595235)
('Percent Discount', -0.19393399357795715)
('Equation Solving Two or Fewer Steps', -0.0024461087596137077)
('Conversion of Fraction Decimals Percents', -0.00019114260794594884)
('Solving for a variable', -0.0011449409066699445)
('Order of Operations +,-,/,* () positive reals', -0.24241811537649482)
('Ordering Positive Decimals', -0.20605534315109253)
('Order of Operations All', -0.00010879390174522996)
('Subtraction Whole Numbers', -0.2424181153